In [3]:
import pickle
from script.util import LSTMModel, rmspe, maape
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import copy
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from random import randint
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.metrics import mean_absolute_error, mean_squared_error
from random import randint

# Use this 
PeMS = pd.read_csv("./data/PEMS04/PeMS04flow.csv",index_col=0)
maximum = PeMS.max().max()

In [107]:
filelist =["newS1.pkl","clusterS5.pkl","clusterS10.pkl"]

[1, [2, 3]]

In [105]:
def my_metric_dict(file,metric=rmse):
    #Load the best model and evaluate on the test set
    with open('./experiment/'+file, 'rb') as f:
                dict = pickle.load(f)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    criterion = torch.nn.MSELoss()
    size = dict["size"]    
    dictrmse = {}
    dictrmspe = {}
    for s in range(95+size):
        if s =="size":
            continue
        dictrmse[s] = []
        dictrmspe[s]=[]
    for s in range(80):
        if s =="size":
            continue
        best_model = dict[s]["model"]
        best_model.double()
        device = torch.device('cuda:0')
        best_model.to(device)
        test_loader = dict[s]["test"]
        best_model.eval()

        # Evaluate the model on the test set
        test_loss = 0.0
        predictions = []
        actuals = []

        with torch.no_grad():
            for i, (inputs, targets) in enumerate(test_loader):
                inputs = inputs.to(device)
                targets = targets.to(device)
                x = torch.Tensor(inputs).unsqueeze(1).to(device)
                y = torch.Tensor(targets).unsqueeze(0).to(device)
                outputs = best_model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
                # Save the predictions and actual values for plotting later
                predictions.append(outputs.cpu().numpy())
                actuals.append(targets.cpu().numpy())
                x            
        test_loss /= len(test_loader)
        # Concatenate the predictions and actuals

        predictions = np.concatenate(predictions, axis=0)
        actuals = np.concatenate(actuals, axis=0)
        EPSILON = 1e-10
        metric_dict_multi = {}
        for k in range(size): 
            y_pred = predictions[:,k]*maximum
            y_true = actuals[:,k]*maximum
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            mae = mean_absolute_error(y_true, y_pred)
            mape = mean_absolute_percentage_error(y_true,y_pred+EPSILON)
            maape =  np.mean(np.arctan(np.abs((y_true - y_pred) / (y_true + EPSILON))))*100
            rmspeval = rmspe(y_true,y_pred)
            #print(s+k,' : ', rmse, maape)
            dictrmse[s+k].append(rmspeval)
            number_of_zero = len(y_true[y_true ==0])
    return dictrmse 



In [93]:
file ="newS1.pkl"
rmspe1 = my_metric_dict(file)


In [94]:
file ="clusterS5.pkl"
rmspe5 = my_metric_dict(file)



In [95]:
file ="clusterS10.pkl"
rmspe10 = my_metric_dict(file)

In [90]:
allrmse = {}
for i in range(80):
    allrmse[i]= rmse1[i]+ rmse5[i] + rmse10[i]

In [77]:
for i in range(80):
    if allrmse[i].index(min(allrmse[i])) != 0 : 
        print('For sensor {} RMSE multivarié indice {} : {}'.format(i,allrmse[i].index(min(allrmse[i])),min(allrmse[i])))
        print('RMSE univarié : {}'.format(allrmse[i][0]))
        print(' ')

For sensor 44 RMSE multivarié indice 5 : 12.619096488287484
RMSE univarié : 12.82454595586525
 
For sensor 45 RMSE multivarié indice 5 : 2.2830519519090324
RMSE univarié : 2.4297174876368075
 
For sensor 46 RMSE multivarié indice 4 : 2.2749591590119884
RMSE univarié : 2.8676495810468317
 
For sensor 47 RMSE multivarié indice 3 : 2.2742494901942387
RMSE univarié : 2.402942770296334
 
For sensor 48 RMSE multivarié indice 2 : 2.2991594667271533
RMSE univarié : 2.7251702046132023
 
For sensor 49 RMSE multivarié indice 1 : 2.2736563530308773
RMSE univarié : 2.654160662823519
 


In [103]:
allrmspe = {}
for i in range(80):
    allrmspe[i]= rmspe1[i]+ rmspe5[i] + rmspe10[i]
for i in range(43,50):    
    if allrmspe[i].index(min(allrmspe[i])) != 0 : 
        print('For sensor {} RMSE multivarié indice {} : {:.4f} %'.format(i,allrmspe[i].index(min(allrmspe[i])),min(allrmse[i])))
        print('RMSE univarié : {:.4f} %'.format(allrmspe[i][0]))
        print(' ')

For sensor 44 RMSE multivarié indice 5 : 12.6191 %
RMSE univarié : 155.1325 %
 
For sensor 45 RMSE multivarié indice 5 : 2.2831 %
RMSE univarié : 2.3286 %
 
For sensor 46 RMSE multivarié indice 4 : 2.2750 %
RMSE univarié : 3.3253 %
 
For sensor 48 RMSE multivarié indice 2 : 2.2992 %
RMSE univarié : 2.9554 %
 
For sensor 49 RMSE multivarié indice 1 : 2.2737 %
RMSE univarié : 3.2042 %
 
